In [1]:
using Revise

In [11]:
using Flux
using JLD2
using FileIO
using MLDataPattern
using CoordinateTransformations
using ProgressMeter
using RigidBodyDynamics
using Gurobi
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
import FluxExtensions
import LearningMPC
import LCPSim
import Hoppers

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/Hoppers.ji for module Hoppers.


In [4]:
samples = load("2017-12-20-hopper-grid/grid_search.jld2")["samples"];

In [9]:
minimum(samples) do sample
    sample.mip.objective_bound
end

2.2224779361445144

In [8]:
maximum(samples) do sample
    sample.mip.objective_value
end

1459.1910382673889

In [10]:
samples[1]

LearningMPC.Sample{Float64}([0.25, 0.25, -5.0, -5.0], [-0.0 -3.44884e-13 … 7.031e-15 -1.62567e-21; 40.0 -5.19332e-13 … 8.27324e-15 9.80654e-27], [445.908], LearningMPC.MIPResults
  solvetime_s: Float64 0.0306549072265625
  objective_value: Float64 232.33189199391802
  objective_bound: Float64 210.50321396361096
)

In [12]:
robot = Hoppers.Hopper()
xstar = Hoppers.nominal_state(robot)
ustar = zeros(num_velocities(xstar))
basevis = Visualizer()[:hopper]
setgeometry!(basevis, robot)
settransform!(basevis[:robot], xstar)

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
Δt = 0.05
Jc = LCPSim.ContactLQR.contact_jacobian(xstar, [Point3D(default_frame(foot), 0., 0., 0.)])
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)

([0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [0.0 0.0; 0.0 0.0; 0.0 1.0; 0.0 1.0], [0.0, 0.0, -9.81, -9.81])

In [16]:
features(sample::LearningMPC.Sample) = (sample.state, sample.mip.objective_bound, sample.mip.objective_value, sample.uJ[:, 1])
data = features.(samples);
train_data, test_data = splitobs(shuffleobs(data), at=0.85);

In [33]:
function setup_model(R, B)
    model = FluxExtensions.TangentPropagator(Chain(
        LinearMap(UniformScaling(0.2)),
        Dense(4, 32, elu),
        Dense(32, 32, elu),
        Dense(32, 1, elu),
        AffineMap(fill(750, 1, 1), [750])
        ))
    RiBt = inv(R) * B' 

    function sample_loss(w_tangent)
        (x, lb, ub, input) -> begin
            y, J = model(x)
            value_cost = sum(ifelse.(y .< lb, lb .- y, ifelse.(y .> ub, y .- ub, 0 .* y)))
            tangent_cost = w_tangent * sum(abs2.(-RiBt * J' .- input))
            value_cost + tangent_cost
        end
    end
    function batch_loss(w_tangent)
        loss = sample_loss(w_tangent)
        (samples) -> sum((sample -> loss(sample...)).(samples)) / length(samples)
    end
    model, batch_loss
end

model, loss = setup_model(R, B)
opt = Flux.ADADelta(params(model))

(::#71) (generic function with 1 method)

In [38]:
loss(0.0)(train_data[1:1000])

Tracked 0-dimensional Array{Float64,0}:
551.6

In [39]:
train_loss = loss(1.0)
@showprogress for i in 1:200
    batches = eachbatch(shuffleobs(train_data), size=50)
    for batch in batches
        l = train_loss(batch)
        isinf(Flux.Tracker.value(l)) && error("Loss is Inf")
        isnan(Flux.Tracker.value(l)) && error("Loss is NaN")
        Flux.back!(l)
        opt()
    end
    @show loss(1.0)(train_data) loss(0.0)(train_data)
end

(loss(1.0))(train_data) = param(6.87789e5)
(loss(0.0))(train_data) = param(479.016)


Progress:   0%|                                         |  ETA: 15:26:52

(loss(1.0))(train_data) = param(9846.29)
(loss(0.0))(train_data) = param(169.417)


Progress:   1%|                                         |  ETA: 15:22:47

(loss(1.0))(train_data) = param(291.665)
(loss(0.0))(train_data) = param(227.037)


...

Progress:  99%|█████████████████████████████████████████|  ETA: 0:09:05

(loss(1.0))(train_data) = param(127.183)
(loss(0.0))(train_data) = param(108.805)


Progress: 100%|█████████████████████████████████████████|  ETA: 0:04:32

(loss(1.0))(train_data) = param(135.504)
(loss(0.0))(train_data) = param(108.217)


Progress: 100%|█████████████████████████████████████████| Time: 15:08:19


In [40]:
jldopen("2018-01-25-value-tangents-data.jld2", "w") do file
    file["model"] = model
    file["params"] = params(model)
end

6-element Array{Any,1}:
 param([-0.874318 -0.894906 0.449782 -0.29218; -0.0491876 -0.916839 0.0816046 -0.0822522; … ; -0.753808 -2.2484 0.0164334 -0.0220471; -0.135119 -0.0400157 0.206078 -0.206259])                                              
 param([-0.685669, -0.508003, 0.0230763, 0.150286, -0.280697, -0.0709213, 0.370353, 0.528606, -0.17281, -1.45689  …  -0.12054, 0.0365872, 0.5142, 0.171768, -1.13385, 0.0225197, -1.77688, -0.431941, -0.724202, -0.0487545])
 param([0.0827438 0.181199 … 0.0714366 0.429206; 0.296025 0.318421 … 0.190893 0.398564; … ; -0.350043 -0.300057 … -0.0530053 -0.973669; 0.331874 0.488173 … 0.620462 0.636777])                                              
 param([-0.876633, -0.984719, -0.130419, -1.70156, 1.24424, -0.45569, 0.0369737, -1.02634, -1.16589, -0.681811  …  -0.480377, -1.24427, -1.37558, -0.426286, -1.05001, -0.729481, -0.587845, 2.35657, 1.83911, -0.917382])   
 param([0.0186335 0.091613 … 0.00885737 0.00890532])                                    

In [45]:
import LCPSim
import Hoppers
using RigidBodyDynamics
using Gurobi
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

Process(`/home/rdeits/locomotion/explorations/learning-mpc/packages/v0.6/DrakeVisualizer/src/../deps/usr/bin/drake-visualizer`, ProcessRunning)

In [46]:
robot = Hoppers.Hopper()
xstar = Hoppers.nominal_state(robot)
ustar = zeros(num_velocities(xstar))
basevis = Visualizer()[:hopper]
setgeometry!(basevis, robot)
settransform!(basevis[:robot], xstar)

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
Δt = 0.05
Jc = LCPSim.ContactLQR.contact_jacobian(xstar, [Point3D(default_frame(foot), 0., 0., 0.)])
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)

([0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [0.0 0.0; 0.0 0.0; 0.0 1.0; 0.0 1.0], [0.0, 0.0, -9.81, -9.81])

In [47]:
model(zeros(4))

(param([223.087]), param([-32.3593 3.97868 -0.706242 0.638184]))

In [48]:
net_value_controller = state -> begin
    x = state_vector(state)
    value, jac = model(x)
    u = vec(-inv(R) * B' * Flux.Tracker.value(jac)')
end

(::#99) (generic function with 1 method)

In [51]:
x_init = MechanismState{Float64}(robot.mechanism)
set_configuration!(x_init, [1.0, 1.0])
set_velocity!(x_init, [0., 0.])
LearningMPC.randomize!(x_init, x_init, 0.5, 1.0)
results = LCPSim.simulate(x_init, net_value_controller,
    robot.environment,
    Δt,
    100,
    GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [52]:
LearningMPC.playback(basevis[:robot], results, Δt)